# 文書生成AIのファインチューニング・レシピ
本Notebookでは、MARC-jaというAmazonのレビューデータセットに含まれる文の一対比較によって報酬モデルを訓練するコードを記載します。

## 応用レシピ：報酬モデルの訓練

### (1) ライブラリのインストール

In [1]:
%pip install transformers==4.35.2
%pip install trl[peft]==0.7.10
%pip install wandb==0.16.2
%pip install sentencepiece==0.1.99
%pip install accelerate==0.26.1
%pip install bitsandbytes==0.42.0
%pip install datasets==2.16.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 94.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━

### (2) Weights & Biases及びHugging Face Hubへのログイン
学習時のログを記録するWeights & Biases、及び訓練後のモデルの保存先であるHuggingface Hubにログインします。いずれも事前にアカウントの作成とトークンの取得が必要です。

In [2]:
import wandb
wandb.init()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
from huggingface_hub import notebook_login
notebook_login()

### (3) ライブラリのインポート

In [4]:
import torch
from tqdm import tqdm
import numpy as np
import pandas as pd

tqdm.pandas()

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from peft import LoraConfig, TaskType
from trl import RewardTrainer, RewardConfig

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


### (4) データセットの準備

In [5]:
base_model_name = 'rinna/japanese-gpt2-medium'
reward_model_name = 'taku-yoshioka/reward-model-0828'
dataset_name, subset_name = 'shunk031/JGLUE', 'MARC-ja'

In [6]:
ds = load_dataset(dataset_name, split='train', name=subset_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for shunk031/JGLUE contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/shunk031/JGLUE
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.w

/root/.cache/huggingface/modules/datasets_modules/datasets/shunk031--JGLUE/bef660b0971cf102c95d2554bd2552ce5fb1cf8e9223cc6f8001c70be571c18c/JGLUE.py:464: UserWarning: can't import `mojimoji`, failing back to method that do nothing. We recommend running `pip install mojimoji` to reproduce the original preprocessing.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [7]:
type(ds), len(ds)

(datasets.arrow_dataset.Dataset, 187528)

`RewardTrainer`クラスが利用可能なデータセットを作成します。

In [8]:
def preprocess(dataset, tokenizer, n_pairs):
    # positiveとnegativeに分割
    pos = []
    neg = []
    for item in tqdm(ds):
        if item['label'] == 0:
            pos.append(item['sentence'])
        else:
            neg.append(item['sentence'])
    print("Num of samples pos {}: neg {}".format(len(pos), len(neg)))

    # トーカナイズ
    pos_ids, pos_masks = [], []
    for text in tqdm(pos):
        tokenized = tokenizer(text, return_tensors='pt', truncation=True)
        pos_ids.append(tokenized['input_ids'].squeeze(0))
        pos_masks.append(tokenized['attention_mask'].squeeze(0))

    neg_ids, neg_masks = [], []
    for text in tqdm(neg):
        tokenized = tokenizer(text, return_tensors='pt', truncation=True)
        neg_ids.append(tokenized['input_ids'].squeeze(0))
        neg_masks.append(tokenized['attention_mask'].squeeze(0))

    # サンプリング
    rng = np.random.default_rng(seed=42)
    ixs_pos = rng.choice(len(pos), size=n_pairs)
    ixs_neg = rng.choice(len(neg), size=n_pairs)

    examples = []

    for ix_pos, ix_neg in zip(ixs_pos, ixs_neg):
        examples.append({
            'input_ids_chosen': pos_ids[ix_pos],
            'attention_mask_chosen': pos_masks[ix_pos],
            'input_ids_rejected': neg_ids[ix_neg],
            'attention_mask_rejected': neg_masks[ix_neg],
        })

    return examples


In [9]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name, model_max_length=512)
tokenizer.pad_token = tokenizer.eos_token
dataset = preprocess(ds, tokenizer, n_pairs=180000) # 元のデータセットに含まれる文書数と大体同じ数の組とする

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/282 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/806k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/153 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
100%|██████████| 187528/187528 [00:06<00:00, 29035.71it/s]


Num of samples pos 165477: neg 22051


100%|██████████| 22051/22051 [00:10<00:00, 2056.37it/s]


### (5) 報酬モデルの訓練

In [10]:
model = AutoModelForSequenceClassification.from_pretrained(
    base_model_name, num_labels=1
)
model.config.pad_token_id = tokenizer.eos_token_id

peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    # inference_mode=False,
    inference_mode=True,
    bias="none",
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)

reward_config = RewardConfig(
    output_dir="output",
    per_device_train_batch_size=64,
    num_train_epochs=5,
    gradient_accumulation_steps=16,
    learning_rate=1.41e-3,
    report_to="wandb",
    remove_unused_columns=False,
    optim="adamw_torch",
    logging_strategy="steps",
    logging_steps=1,
    max_length=512,
    seed=42,
)

trainer = RewardTrainer(
    model=model,
    args=reward_config,
    tokenizer=tokenizer,
    train_dataset=dataset,
    peft_config=peft_config,
)

trainer.train()

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


model.safetensors:   0%|          | 0.00/1.37G [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at rinna/japanese-gpt2-medium and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1091: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2632: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
`use_cache=True` is incompatible with gradient ch

Step,Training Loss
1,0.653000
2,0.607800
3,0.548500
4,0.490300
5,0.441100
6,0.417600
7,0.384500
8,0.351900
9,0.313900
10,0.318300


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2632: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


TrainOutput(global_step=875, training_loss=0.12030494185856411, metrics={'train_runtime': 21706.7427, 'train_samples_per_second': 41.462, 'train_steps_per_second': 0.04, 'total_flos': 0.0, 'train_loss': 0.12030494185856411, 'epoch': 4.98})

### (6) 報酬モデルの保存

In [11]:
trainer.model.push_to_hub(reward_model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/taku-yoshioka/reward-model-0828/commit/d44de852058b444890caac227eb8c92f1b23c8ee', commit_message='Upload model', commit_description='', oid='d44de852058b444890caac227eb8c92f1b23c8ee', pr_url=None, pr_revision=None, pr_num=None)